In [20]:
import yaml

In [4]:
with open('schema.yaml', 'r') as file:
    semantic_model = yaml.load(file, Loader=yaml.FullLoader)

# Extract tables and columns


In [5]:
from langchain_core.prompts import ChatPromptTemplate
question="What was the total revenue from North America for each product category in 2023?"
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
You are an intelligent assistant that generates most efficient sql queries using semantic model. Use the semantic model below to interpret the query and generate accurate SQL statements. The output should only give the most optimized sql query

{semantic_model}"""
),
    ("user", """Respond to the query : {question}""")
])

In [6]:
OPENAI_API_KEY = "f29d9eef53fd451388bd7b1bc1a062a7"
OPENAI_DEPLOYMENT_NAME = "gpt-4"
AZURE_OPENAI_ENDPOINT = "https://deployment-gpt4.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2024-08-01-preview"

In [7]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableSequence
llm_client=AzureChatOpenAI(
    api_key=OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_deployment=OPENAI_DEPLOYMENT_NAME
    
)

In [8]:
from langchain_core.output_parsers import StrOutputParser
chain=prompt_template | llm_client | StrOutputParser()


In [ ]:
chain.invoke({"semantic_model":fr"""{semantic_model}""","question":question})


In [ ]:
type(prompt_template.invoke({"semantic_model":fr"""{semantic_model}""","question":question}))

In [11]:
import os

In [ ]:
os.curdir

In [90]:
all_data={}
for yaml_files in os.listdir(os.curdir):
    if yaml_files.endswith(".yaml"):
        with open(yaml_files,'r') as fp:
            yaml_data=yaml.load(fp, Loader=yaml.FullLoader)
            all_data[fp.name]=yaml_data
            fp.close()
    

In [ ]:
all_data

In [ ]:
import os, yaml
data=[]
for yaml_files in os.listdir(os.curdir):
        if yaml_files.endswith(".yaml"):
            print(yaml_files)
            with open(yaml_files,'r',encoding='utf-8') as fp:
                yaml_data=yaml.safe_load(fp)
                document_data={}
                for table in yaml_data['tables']:
                    document_data={
                        "table":{
                                            "table_name": table['name'],
                                            "table_description": table['description'],
                                   },
                                            "base_table":{
                                                "database": table['base_table']['database'],
                                                "schema": table['base_table']['schema'],
                                   }
                            }
                    for column in table['dimensions']:
                        document_data['columns']={
                            "column_name": column['name'],
                            "column_desc": column['description'],
                            "column_type": column['data_type'],
                            "column_expr": column["expr"],
                            "column_sample_values": column["sample_values"] if "sample_values" in column else None,
                            }
                            
                        
                        
                    
                                            
                        
                                    
                    
            data.append(document_data)        

In [91]:
import datetime
data=[]
c=1
for yaml_files in os.listdir(os.curdir):
    if yaml_files.endswith(".yaml"):
        print(yaml_files)
        with open(yaml_files,'r',encoding='utf-8') as fp:
            yaml_data=yaml.safe_load(fp)
            document_data=json.dumps(yaml_data, indent=4)
            document_data=json.loads(document_data)
            document_data["id"]=str(c)+datetime.datetime.now().strftime(r"%Y%m%d%H%M%S")
            c+=1
            data.append(document_data)            

semantic_model (13).yaml
semantic_model (14).yaml
semantic_model (15).yaml
semantic_model (16).yaml
semantic_model (17).yaml
semantic_model (18).yaml
semantic_model (19).yaml
semantic_model (20).yaml
semantic_model (21).yaml
semantic_model (22).yaml
semantic_model (23).yaml
semantic_model (24).yaml
semantic_model (25).yaml
semantic_model (26).yaml
semantic_model (27).yaml
semantic_model (28).yaml
semantic_model (29).yaml
semantic_model (30).yaml


In [88]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

In [98]:
data[0]

{'name': 'yaml1  ',
 'tables': [{'name': 'AIRCRAFT_CARRIER_INDEX',
   'description': 'The Aircraft Carrier Index table provides a list of airline carriers, including their unique identification codes, names, and key operational details.',
   'base_table': {'database': 'GLOBAL_GOVERNMENT',
    'schema': 'CYBERSYN',
    'table': 'AIRCRAFT_CARRIER_INDEX'},
   'dimensions': [{'name': 'AIRCRAFT_CARRIER_ID',
     'synonyms': ['  '],
     'description': 'Unique ID for the aircraft carrier.',
     'expr': 'AIRCRAFT_CARRIER_ID',
     'data_type': 'TEXT',
     'sample_values': ['01228_62', '06932_11', '01145_62']},
    {'name': 'CARRIER_NAME',
     'synonyms': ['  '],
     'description': 'Name of the airport carrier.',
     'expr': 'CARRIER_NAME',
     'data_type': 'TEXT',
     'sample_values': ['USA Jet Airlines Inc. (1994 - )',
      'Sportsflight Airways Inc. (1994 - 1996)',
      'Pacific International Airlines Inc. (1996 - 1996)']},
    {'name': 'CARRIER_TYPE',
     'synonyms': ['  '],
    

In [96]:
service_name = 'gppg-ai-service'
admin_key = "sDbfqRtI0rfK9bIMh5faA54lAxDaMBABSA3uM0r7YLAzSeCk3KcD"
index_name = 'pop-index'
endpoint = f"https://{service_name}.search.windows.net/"


search_client = SearchClient(
    endpoint=endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
    )

res=search_client.upload_documents(documents=data[0])
print(res)

In [99]:
for doc in data:
    try:
        res=search_client.upload_documents(documents=doc)
        print(res)
    except Exception as e:
        print(e)

() The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
Code: 
Message: The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
() The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
Code: 
Message: The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
() The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
Code: 
Message: The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
() The request is invalid. Details: Cannot find nested property 'time_dimensions' on the resource type 'search.complex.tables'.
Code: 
Message: The request is invalid. Details: Cannot find nest

In [2]:
import snowflake.connector as sf
import os

In [3]:
SNOWFLAKE_ROLE=os.getenv("SNOWFLAKE_ROLE")
SNOWFLAKE_WAREHOUSE=os.getenv("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_USER=os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_ACCOUNT_LOCATOR=os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_HOST=os.getenv("SNOWFLAKE_HOST")
SNOWFLAKE_PASSWORD=os.getenv("SNOWFLAKE_PASSWORD")

In [6]:
conn = sf.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT_LOCATOR,
        warehouse=SNOWFLAKE_WAREHOUSE,
        role=SNOWFLAKE_ROLE,
        database='GLOBAL_GOVERNMENT',
        schema=['INFORMATION_SCHEMA','CYBERSYN']

        
        
        
    )

In [7]:
query='''-- Get tables in CYBERSYN schema
WITH TABLES AS (
    SELECT 
        TABLE_SCHEMA,
        TABLE_NAME,
        TABLE_OWNER,
        TABLE_TYPE,
        IS_TRANSIENT,
        CLUSTERING_KEY,
        ROW_COUNT,
        BYTES,
        RETENTION_TIME,
        COMMENT
    FROM GLOBAL_GOVERNMENT.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'CYBERSYN'
),

-- Get columns for CYBERSYN tables
COLUMNS AS (
    SELECT 
        TABLE_SCHEMA,
        TABLE_NAME,
        COLUMN_NAME,
        ORDINAL_POSITION,
        IS_NULLABLE,
        DATA_TYPE,
        CHARACTER_MAXIMUM_LENGTH,
        NUMERIC_PRECISION,
        NUMERIC_SCALE,
        COMMENT,
        IDENTITY_START,
        IDENTITY_INCREMENT
    FROM GLOBAL_GOVERNMENT.INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = 'CYBERSYN'
)

-- Final output combining table and column information
SELECT 
    'GLOBAL_GOVERNMENT' as DATABASE_NAME,
    'CYBERSYN' as SCHEMA_NAME,
    t.TABLE_NAME,
    t.TABLE_OWNER,
    t.TABLE_TYPE,
    t.IS_TRANSIENT as TABLE_IS_TRANSIENT,
    t.CLUSTERING_KEY,
    t.ROW_COUNT,
    ROUND(t.BYTES / (1024*1024), 2) as SIZE_MB,
    t.RETENTION_TIME,
    t.COMMENT as TABLE_COMMENT,
    c.COLUMN_NAME,
    c.ORDINAL_POSITION,
    c.IS_NULLABLE,
    c.DATA_TYPE,
    c.CHARACTER_MAXIMUM_LENGTH,
    c.NUMERIC_PRECISION,
    c.NUMERIC_SCALE,
    c.IDENTITY_START,
    c.IDENTITY_INCREMENT,
    c.COMMENT as COLUMN_COMMENT
FROM TABLES t
LEFT JOIN COLUMNS c ON t.TABLE_NAME = c.TABLE_NAME
ORDER BY 
    t.TABLE_NAME,
    c.ORDINAL_POSITION;'''

In [9]:
curr=conn.cursor()
            
res=curr.execute(query)

In [10]:
curr.fetchall()

[('GLOBAL_GOVERNMENT',
  'CYBERSYN',
  'AIRCRAFT_CARRIER_INDEX',
  None,
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'The Aircraft Carrier Index table provides a list of airline carriers, including their unique identification codes, names, and key operational details.',
  'AIRCRAFT_CARRIER_ID',
  1,
  'YES',
  'TEXT',
  16777216,
  None,
  None,
  None,
  None,
  'Unique ID for the aircraft carrier.'),
 ('GLOBAL_GOVERNMENT',
  'CYBERSYN',
  'AIRCRAFT_CARRIER_INDEX',
  None,
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'The Aircraft Carrier Index table provides a list of airline carriers, including their unique identification codes, names, and key operational details.',
  'CARRIER_NAME',
  2,
  'YES',
  'TEXT',
  16777216,
  None,
  None,
  None,
  None,
  'Name of the airport carrier.'),
 ('GLOBAL_GOVERNMENT',
  'CYBERSYN',
  'AIRCRAFT_CARRIER_INDEX',
  None,
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'The Aircraft Carrier Index table provides a list of airline 